## Datebase Statistics Example
This notebooks shows a couple of basic examples about how to create statistics from details stored in your Home Assistant database. 

### Setup Database connection

In [1]:
# Instead of SQLAlchemy is the built-in Python support for SQLite used.
import sqlite3

In [2]:
# Your database URL as specified in configuration.yaml
DB_URL = "/path/to/.homeassistant/home-assistant_v2.db"

In [3]:
conn = sqlite3.connect(DB_URL)

### Details about your entities

Get all entities and their occurrence from your database. Limit to 10.

In [4]:
entities = conn.execute("SELECT entity_id, COUNT(*) as count FROM states "
                        "GROUP BY entity_id ORDER BY count DESC LIMIT 10")

In [5]:
entities.fetchall()

[('sensor.random_sensor', 25583),
 ('sensor.cpu_speed', 17963),
 ('sun.sun', 17655),
 ('binary_sensor.threshold', 3565),
 ('sensor.random', 3475),
 ('sensor.cpu', 2730),
 ('automation.check_sensor_value_and_show_notification_2', 2566),
 ('sensor.time', 1967),
 ('sensor.new_york', 1926),
 ('sensor.battery', 1688)]

### Details about one entities

Most SQL dialects support mathematical functions, especially for statistical analysis. For SQLite those are called [Aggregate Functions](https://www.sqlite.org/lang_aggfunc.html).

In [6]:
stats = conn.execute("SELECT entity_id, COUNT(state) as count, MAX(state) as max, "
                     "MIN(state) as min, AVG(state) as mean FROM states "
                     "WHERE entity_id = 'sensor.cpu'")

In [7]:
for row in stats:
   print("Entitiy ID : ", row[0])
   print("count      : ", row[1])
   print("max        : ", row[2])
   print("min        : ", row[3])
   print("mean       : ", row[4])

Entitiy ID :  sensor.cpu
count      :  2730
max        :  2.8
min        :  0.47
mean       :  1.2027509157509158


Count the occurrence of a single values. Could be useful if you want to adjust your climate units.

In [8]:
stats1 = conn.execute("SELECT entity_id, COUNT(state) FROM states "
                      "WHERE entity_id = 'sensor.cpu' AND state = 2.3")

In [9]:
stats1.fetchall()

[('sensor.cpu', 12)]

Show only values which exceeded a threshold.

In [10]:
stats2 = conn.execute("SELECT entity_id, COUNT(state) FROM states "
                      "WHERE entity_id = 'sensor.cpu' AND state > 2.3")

In [11]:
stats2.fetchall()

[('sensor.cpu', 378)]

Determine the states of a binary sensor.

In [12]:
stats3 = conn.execute("SELECT state, COUNT(state) FROM states "
                      "WHERE entity_id = 'binary_sensor.movement' "
                      "AND state = 'off' OR state = 'on' GROUP BY state")

In [13]:
stats3.fetchall()

[('on', 6607)]

If you are a Database or SQL pro, please feel free to add your queries or create a new notebook.